## Setup

In [2]:
source("/research/lab_reiberger/2022_PSVD_metabolomics/2022_PSVD_metabolomics/R/00_setup.R")

In [3]:
p_load("dplyr", "ggplot2", "mice", "janitor", "POMA", "SummarizedExperiment", install = FALSE)

In [4]:
lipid_raw <- read.csv("data/lipid.csv", fileEncoding="windows-1252", check.names = FALSE, na.strings="NA")
facid_raw <- read.csv("data/facid.csv", fileEncoding="windows-1252", check.names = FALSE, na.strings="NA")
metab_raw <- read.csv("data/metab.csv", fileEncoding="windows-1252", check.names = FALSE, na.strings="NA")

In [9]:
metadata <- facid_raw[, 3:7]
names(metadata) <- make_clean_names(names(metadata))
metadata <- metadata %>% dplyr::rename("sample_id" = "sample_identification")

## Data cleaning

In [11]:
missing_threshold <- 20

In [12]:
lipid_raw_na <- lipid_raw %>% purrr::discard(~sum(is.na(.x))/length(.x)* 100 >= missing_threshold)
paste0("Columns with >", missing_threshold, "% missing values in lipid_raw: ", ncol(lipid_raw) - ncol(lipid_raw_na), ". New number of columns: ", ncol(lipid_raw_na), ". Percentage of outliers: ", round((ncol(lipid_raw) - ncol(lipid_raw_na)) * 100/ncol(lipid_raw), 2), "%")

facid_raw_na <- facid_raw %>% purrr::discard(~sum(is.na(.x))/length(.x)* 100 >= missing_threshold)
paste0("Columns with >", missing_threshold, "% missing values in facid_raw: ", ncol(facid_raw) - ncol(facid_raw_na), ". New number of columns: ", ncol(facid_raw_na), ". Percentage of outliers: ", round((ncol(facid_raw) - ncol(facid_raw_na)) * 100/ncol(facid_raw), 2), "%")

metab_raw_na <- metab_raw %>% purrr::discard(~sum(is.na(.x))/length(.x)* 100 >= missing_threshold)
paste0("Columns with >", missing_threshold, "% missing values in metab_raw: ", ncol(metab_raw) - ncol(metab_raw_na), ". New number of columns: ", ncol(metab_raw_na), ". Percentage of outliers: ", round((ncol(metab_raw) - ncol(metab_raw_na)) * 100/ncol(metab_raw), 2), "%")

[1] "Columns with >20% missing values in lipid_raw: 95. New number of columns: 197. Percentage of outliers: 32.53%"

[1] "Columns with >20% missing values in facid_raw: 13. New number of columns: 31. Percentage of outliers: 29.55%"

[1] "Columns with >20% missing values in metab_raw: 100. New number of columns: 103. Percentage of outliers: 49.26%"

In [13]:
lipid_raw_na <- remove_constant(lipid_raw_na)
facid_raw_na <- remove_constant(facid_raw_na)
metab_raw_na <- remove_constant(metab_raw_na)

In [14]:
names(lipid_raw_na) <- make_clean_names(names(lipid_raw_na))
names(facid_raw_na) <- make_clean_names(names(facid_raw_na))
names(metab_raw_na) <- make_clean_names(names(metab_raw_na))

In [15]:
lipid_raw_na <- lipid_raw_na %>% select(-sample_id, -sample_description, -group_123, -sex, -age)
facid_raw_na <- facid_raw_na %>% select(-sample_code, -sample_description, -group_123, -sex, -age)
metab_raw_na <- metab_raw_na %>% select(-group_123, -sex, -age)

In [17]:
lipid_raw_na <- lipid_raw_na %>% dplyr::rename("sample_id" = "label")
facid_raw_na <- facid_raw_na %>% dplyr::rename("sample_id" = "sample_identification")
metab_raw_na <- metab_raw_na %>% dplyr::rename("sample_id" = "sample_identification") 

## Imputation, normalization, scaling and outliers

In [31]:
lipid_se <- PomaSummarizedExperiment(target = metadata, features = lipid_raw_na[2:ncol(lipid_raw_na)])
facid_se <- PomaSummarizedExperiment(target = metadata, features = facid_raw_na[2:ncol(facid_raw_na)])
metab_se <- PomaSummarizedExperiment(target = metadata, features = metab_raw_na[2:ncol(metab_raw_na)])


In [40]:
lipid_se <- PomaImpute(lipid_se, ZerosAsNA = TRUE, cutoff = 20, method = "knn") %>% PomaNorm(method = "log_pareto") %>% PomaOutliers(coef = 3)
facid_se <- PomaImpute(facid_se, ZerosAsNA = TRUE, cutoff = 20, method = "knn") %>% PomaNorm(method = "log_pareto") %>% PomaOutliers(coef = 3)
metab_se <- PomaImpute(metab_se, ZerosAsNA = TRUE, cutoff = 20, method = "knn") %>% PomaNorm(method = "log_pareto") %>% PomaOutliers(coef = 3)

ERROR: Error in PomaImpute(lipid_se, ZerosAsNA = TRUE, cutoff = 20, method = "knn"): data is not a SummarizedExperiment object. 
See POMA::PomaSummarizedExperiment or SummarizedExperiment::SummarizedExperiment


In [33]:
lipid_se <- as.data.frame(t(assay(lipid_se))) %>% tibble::rownames_to_column("sample_id")
facid_se <- as.data.frame(t(assay(facid_se))) %>% tibble::rownames_to_column("sample_id")
metab_se <- as.data.frame(t(assay(metab_se))) %>% tibble::rownames_to_column("sample_id")

## Integration

In [34]:
metabolomics_merged <- merge(lipid_se, facid_se, by="sample_id")
metabolomics_merged <- merge(metabolomics_merged, metab_se, by = "sample_id")

In [41]:
metabolomics_merged

sample_id,ce_16_1,ce_18_0,ce_18_1,ce_18_2,ce_18_3,ce_20_3,ce_20_4,ce_22_6,cer_d34_1,⋯,tyrosine,uracil,uric_acid,uridine,uridine_diphosphohexose,uridine_monophosphate,valine,xanthine,xanthosine,xylitol
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2341,0.762,0.863,0.451,-1.220,0.016,-1.810,-0.412,-0.309,0.012,⋯,0.168,0.710,0.414,1.061,0.136,0.806,-0.443,0.149,-0.029,-0.059
2344,-0.525,-0.411,0.036,-0.952,-0.224,-1.181,-1.596,-1.185,0.283,⋯,0.725,0.285,0.119,0.381,-0.032,0.199,-0.087,0.134,0.015,0.068
2358,-0.555,-0.032,-0.041,-0.295,-0.331,-0.563,-1.474,-1.091,0.262,⋯,-0.469,0.048,-0.301,0.493,0.052,0.147,-0.148,1.343,0.146,-0.212
2360,0.156,0.453,-0.095,-0.440,-0.051,-0.502,-0.936,-0.863,0.462,⋯,0.318,-0.150,0.165,-0.564,0.010,0.094,-0.166,-0.380,-0.029,-0.010
2390,0.243,0.050,0.313,-0.358,-0.304,-0.437,-0.837,-0.795,0.164,⋯,-0.702,0.285,-0.713,-0.061,0.136,0.326,-0.019,0.299,-0.029,-0.226
2427,-0.833,-0.297,-0.550,-0.678,-0.449,-1.106,-0.485,-0.564,-0.031,⋯,-0.392,0.075,0.135,0.093,0.010,0.199,-0.314,0.214,-0.029,-0.334
2432,-0.169,-0.904,-0.459,-0.801,-0.286,-0.722,-0.910,-0.972,0.152,⋯,-0.585,0.021,0.551,-0.110,-0.032,0.040,-0.539,0.139,0.059,0.703
2486,0.005,-0.963,-0.395,-0.086,-0.729,-0.071,-0.104,-0.057,-0.091,⋯,0.211,-0.064,0.127,-0.314,-0.076,0.067,0.223,0.234,-0.074,-0.096
2536,-0.289,-0.677,-0.427,-0.444,-0.843,-0.282,-0.211,-0.171,-0.012,⋯,0.216,-0.035,0.375,-0.004,-0.076,0.199,-0.599,0.308,0.189,0.170


In [37]:
write.csv(metabolomics_merged, "outputs/01_metabolomics_merged.csv")

In [38]:
write.csv(metadata, "outputs/01_metadata.csv")